In [2]:
import pandas as pd
import math
import numpy as np
import math
from sklearn.cross_validation import train_test_split # to split the dataset for training and testing

/home/garvit/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [135]:
# getting the ratings data in pandas dataframe
ratings = pd.read_csv('../Yahoo_movies_multi-criteria/data_movies.txt', sep='\t',names=['user_id', 'criterion1', 'criterion2', 'criterion3', 'criterion4', 'overall', 'movie_id', 'num'])
print(max(ratings.overall)) # implies that ratings are on a scale of 13
# train, test = sample_split(ratings)

13


In [65]:
print("users: ",ratings.user_id.drop_duplicates().count()) #number of users
print("movies: ",ratings.movie_id.drop_duplicates().count()) #number of movies

users:  6078
movies:  976


In [66]:
print("minimum movies rated by a user" ,min(ratings.groupby(["user_id"]).overall.count())) # minimum ratings given by any user
print("maximum movies rated by a user" ,max(ratings.groupby(["user_id"]).overall.count())) # maximum ratings given by any user
print("minimum ratings of a movie" ,min(ratings.groupby(["movie_id"]).overall.count())) # minimum ratings of a movie
print("maximum ratings of a movie" ,max(ratings.groupby(["movie_id"]).overall.count())) # maximum ratings of a movie

minimum movies rated by a user 5
maximum movies rated by a user 237
minimum ratings of a movie 1
maximum ratings of a movie 830


In [151]:
# function to crop the dataset such that ratings given by any user>=ru
def min_user_ratings_criteria(ratings, ru):
    # remove all users who have rated <= ru movies
    temp = ratings.groupby(['user_id']).overall.count().reset_index()
#     print(temp)
    users_to_be_removed = temp[temp.overall < ru].user_id.unique()
    print(len(users_to_be_removed))
    new_ratings = ratings[~ratings['user_id'].isin(users_to_be_removed)]
    print(len(new_ratings.user_id.unique()))
    return new_ratings
#     print(len(users_to_be_removed))
# min_ratings_criteria(10,1)

In [153]:
# function to crop the dataset such that every item is rated by atleast ri users
def min_item_ratings_criteria(ratings, ri):
    # remove all items with number of ratings <= ri
    temp = ratings.groupby(['movie_id']).overall.count().reset_index()
#     print(temp)
    items_to_be_removed = temp[temp.overall < ri].movie_id.unique()
    print(len(items_to_be_removed))
    new_ratings = ratings[~ratings['movie_id'].isin(items_to_be_removed)]
    print(len(new_ratings.movie_id.unique()))
    return new_ratings
# min_item_ratings_criteria(5)

In [136]:
# split the datasets into smaller datasets, such that every dataset contains 'size' number of users
def split_dataset(ratings, size):
        df_group = ratings.groupby(['user_id'])
        no_of_datasets = int(len(df_group)/size);
        print(no_of_datasets)
        list_datasets = list() # list containing all the small datasets
        temp = pd.DataFrame()
        count=0
        flag=0
        datasets_generated = 0 # datasets that have been generated till now
        for key, item in df_group:
            count+=1
            temp = temp.append(item)
            if count==size and datasets_generated < no_of_datasets-1:
                list_datasets.append(temp)
                print("size = ",temp.shape[0])
                temp = pd.DataFrame()
                datasets_generated += 1
                count=0
        list_datasets.append(temp) # appending the last dataset
        print("size = ",temp.shape[0])
        return list_datasets
    
# testing above function
# temp = split_dataset(ratings, 500)
# for item in temp:
#     print(len(item.user_id.unique()))

In [137]:
# Spliting a sample into groups userwise
def sample_split(dataFrame):
    df_group = dataFrame.groupby('user_id')
    train = pd.DataFrame()
    test = pd.DataFrame()
    
    for key, item in df_group:
        train, test = split_train_test(item, train, test)
    
    return train, test

In [138]:
# spliting sample in 70% training data and 30% testing data
def split_train_test(dataFrame, train, test):
    temp_train, temp_test = train_test_split(dataFrame, test_size = 0.3, random_state=1212)# in this our main data is split into train and test
    # the attribute test_size=0.3 splits the data into 70% and 30% ratio. train=70% and test=30%
    train = train.append(temp_train)
    test = test.append(temp_test)
    
    return train, test

In [156]:
# remove movies with <10 ratings
rating_dash = min_item_ratings_criteria(ratings, 5)
rating_dash = min_user_ratings_criteria(rating_dash, 10)
print(len(rating_dash.movie_id.unique()))
# list conatining all the datasets
list_datasets = split_dataset(rating_dash,500)
print(rating_dash.shape)

69
907
4371
1707
905
3
size =  10126
size =  10142
size =  14329
(34597, 8)


In [157]:
# splitting into train and test data
train = list()
test = list()
num_of_datasets = len(list_datasets)
print(num_of_datasets)
for i in range(num_of_datasets):
    print(i)
    t1, t2 = sample_split(list_datasets[i])
    train.append(t1)
    test.append(t2)

3
0
1
2


In [13]:
# function to find similarity between two users based on Manhattan distance
def manhattan_similarity(df, user1, user2):
    s = pd.merge(df[df['user_id']==user1], df[df['user_id']==user2], how="inner", on="movie_id")
#     print(s)
    sum1 = 0
    similarity=0
    for index, row in s.iterrows():
        sum1 += abs(row.criterion1_x - row.criterion1_y) + \
                abs(row.criterion2_x - row.criterion2_y) + \
                abs(row.criterion3_x - row.criterion3_y) + \
                abs(row.criterion4_x - row.criterion4_y) + \
                abs(row.overall_x - row.overall_y)
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity
        
# manhattan_similarity(train, 1, 3)

In [15]:
# function to find simialrity between two users based on Euclidean distance
def euclidean_similarity(df, user1, user2):
    s = pd.merge(df[df['user_id']==user1], df[df['user_id']==user2], how="inner", on="movie_id")
#     print(s)
    sum1 = 0
    similarity=0
    for index, row in s.iterrows():
        sum1 += math.sqrt((row.criterion1_x - row.criterion1_y)**2 + \
                (row.criterion2_x - row.criterion2_y)**2 + \
                (row.criterion3_x - row.criterion3_y)**2 + \
                (row.criterion4_x - row.criterion4_y)**2) + \
                (row.overall_x - row.overall_y)**2
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity
# euclidean_similarity(train, 1, 3)

In [16]:
# function to find simialrity between two users based on Chebyshev distance
def chebyshev_similarity(df, user1, user2):
    s = pd.merge(df[df['user_id']==user1], df[df['user_id']==user2], how="inner", on="movie_id")
#     print(s)
    sum1 = 0
    similarity=0
    for index, row in s.iterrows():
        sum1 += max(abs(row.criterion1_x - row.criterion1_y), \
                abs(row.criterion2_x - row.criterion2_y), \
                abs(row.criterion3_x - row.criterion3_y), \
                abs(row.criterion4_x - row.criterion4_y), \
                abs(row.overall_x - row.overall_y))
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity
        
# chebyshev_similarity(train, 1, 3)

In [51]:
# function to predict the rating given by user to item
# neighbours = -1 implies that consider similarity with all possible users
def predict(df, user, item, similarity, neighbours = -1): # df is the train dataset
    neighbours_data_list = list()
    for v in df.user_id.unique():
        if(v==user): # not including the user itself
            continue
        temp = df[df['user_id']==v]
        temp = temp[temp['movie_id']==item]
        if(temp.empty): # user 'v' has not rated the item
            continue
        else:
            rate = temp.iloc[0].overall
        sim = similarity(df, user, v) # find appropriate similarity measure between the two users
        neighbours_data_list.append((sim, rate))
        
    # sort the neighbours_data_list in descending order based on rate
    neighbours_data_list.sort(reverse=True)
    
    # crop the list to the number of neighbours given in the argument
    if(neighbours!=-1):
        length = len(neighbours_data_list)
        neighbours_data_list = neighbours_data_list[:min(neighbours, length)]
    
    # predict the rating using collaborative filtering formula
    numerator = 0
    denominator = 0
    if(len(neighbours_data_list)==0):
        print("We cannot predict")
    for tup in neighbours_data_list:
        numerator += tup[0]*tup[1]
        denominator += tup[0]
    if(denominator==0):
        return 0
    predicted_rating = numerator/denominator
    return predicted_rating
        

In [119]:
# print(test[0].head(26))

In [18]:
# print(train)
print(predict(train[0], 1, 879, euclidean_similarity))
print(predict(train[0], 1, 879, manhattan_similarity))
print(predict(train[0], 1, 879, chebyshev_similarity))

10.37336757999115
9.959976730657361
9.569398389388915


In [19]:
# function to calculate the mean average error between two vectors
def mean_average_error(list1, list2):
    sum1=0
    for a,b in zip(list1,list2):
        if(b==0):            # to handle the unpredicted ratings
            continue
        sum1 += abs(a-b)
    error = sum1/len(list1)
    return error
mean_average_error((1,2),(10,20))

13.5

In [20]:
# function to replace entries in a list with zero/one based on some threshold
def replace_with_zero_one(list1, threshold):
    length = len(list1)
    rlist = list()
    i = 0
    while i < length:
        if(list1[i]==0):   # to handle the unpredicted ratings
            list1[i]=-1
            continue
        if(list1[i]>threshold):
            rlist.append(1)
        else:
            rlist.append(0)
        i += 1
    return rlist

In [21]:
# function to calculate the precision between two lists
# list2 is the predicted list
def precision(list1, list2, threshold): # values > threshold are considered as good values
#     list1 = replace_with_zero_one(list1, threshold)
#     list2 = replace_with_zero_one(list2, threshold)
    tp = 0 # true positives
    fp = 0 # false positives
    for a,b in zip(list1, list2):
        if(a==1 and b==1):
            tp += 1
        if(b==1 and a==0):
            fp+=1
    precision = tp/(tp + fp)
    return precision

# precision([20,5,14], [1,16,20], 13)

In [22]:
# function to calculate the recall between two lists
# list2 is the predicted list
def recall(list1, list2, threshold): # values > threshold are considered as good values
#     list1 = replace_with_zero_one(list1, threshold)
#     list2 = replace_with_zero_one(list2, threshold)
    tp = 0 # true positives
    fn = 0 # false negatives
    for a,b in zip(list1, list2):
        if(a==1 and b==1):
            tp += 1
        if(b==0 and a==1):
            fn += 1
#     print("tp = ",tp)
#     print("fn = ",fn)
    recall = tp/(tp + fn)
    return recall

# recall([20,15,14], [1,16,15], 13)

In [23]:
# function to calculate F1-score between two lists
def f1_score(list1, list2, threshold):
    p = precision(list1, list2, threshold)
    r = recall(list1, list2, threshold)
    f1 = (2*p*r)/(p+r)
    return f1

In [158]:
# predicting on our training dataset
predicted = list() # for storing the predicted ratings
# appending 7 empty lists
predicted.append([])
predicted.append([])
predicted.append([])
predicted.append([])
predicted.append([])
predicted.append([])
predicted.append([])
unpredicted_count = list()
# print(predicted1)

In [ ]:
for i in range(6):
    cnt = 0
    for index, row in test[i].iterrows():
        predicted_rating = predict(train[i], row.user_id, row.movie_id,chebyshev_similarity)
#         print(predicted_rating)
        if(math.isnan(predicted_rating)):
            print("Cannot predict")
            continue
        if(predicted_rating==0):
            cnt += 1
        predicted[i].append(predicted_rating)
        print(predicted_rating)
    print("predicted %d",i)
    unpredicted_count.append(cnt)

In [159]:
# predicting on our training dataset1
print("test-size = ", test[1].shape[0])
predicted[1] = list() # for storing the predicted ratings
i=0
for index, row in test[1].iterrows():
    predicted_rating = predict(train[1], row.user_id, row.movie_id,euclidean_similarity)
    print(i)
    i+=1
    if(math.isnan(predicted_rating) or predicted_rating==0):
        print("Continuing")
        continue
    predicted[1].append(predicted_rating)
print(predicted[1])

test-size =  3247
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
Continuing
23
24
25
26
27
28
29
30
31
We cannot predict
32
Continuing
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
We cannot predict
100
Continuing
101
102
103
We cannot predict
104
Continuing
105
106
107
108
109
110
111
112
113
114
115
Continuing
We cannot predict
116
Continuing
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
We cannot predict
163
Continuing
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
Continuing
186
187
188
189
190
191
192
193
194
Continuing
195
196
197
198
199
200
201
202
203
204
205
206
We cannot predict
207
Continuing
208
209
210
211
212
213
214
215
216
217
21

1571
1572
1573
1574
1575
1576
1577
1578
1579
1580
1581
1582
1583
1584
1585
1586
1587
1588
1589
1590
1591
1592
1593
1594
1595
1596
1597
1598
1599
1600
1601
1602
1603
1604
1605
1606
1607
1608
1609
1610
1611
1612
1613
1614
1615
1616
1617
1618
1619
1620
1621
1622
1623
1624
1625
1626
1627
1628
1629
1630
1631
1632
1633
1634
1635
1636
1637
1638
1639
1640
1641
1642
1643
1644
We cannot predict
1645
Continuing
1646
1647
1648
1649
1650
Continuing
We cannot predict
1651
Continuing
1652
1653
1654
1655
1656
1657
1658
1659
1660
1661
1662
1663
1664
1665
1666
1667
1668
1669
1670
1671
1672
We cannot predict
1673
Continuing
1674
1675
We cannot predict
1676
Continuing
1677
1678
1679
1680
1681
1682
1683
1684
1685
1686
1687
1688
1689
1690
1691
1692
1693
1694
1695
1696
1697
1698
1699
1700
1701
1702
1703
1704
1705
1706
1707
1708
1709
1710
1711
1712
1713
1714
1715
1716
1717
1718
1719
1720
1721
1722
1723
1724
1725
1726
1727
1728
1729
1730
1731
1732
1733
1734
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
174

2989
2990
2991
2992
2993
2994
2995
2996
2997
2998
2999
3000
3001
3002
3003
3004
3005
3006
3007
3008
3009
3010
3011
3012
3013
3014
3015
3016
3017
3018
3019
3020
3021
3022
Continuing
3023
3024
3025
Continuing
3026
3027
3028
3029
3030
3031
3032
3033
3034
3035
3036
3037
3038
3039
3040
3041
3042
3043
3044
3045
3046
3047
3048
3049
3050
3051
3052
3053
3054
3055
3056
3057
3058
3059
3060
3061
3062
3063
3064
3065
3066
3067
3068
3069
3070
3071
3072
3073
3074
3075
3076
3077
3078
3079
3080
3081
3082
3083
3084
3085
3086
3087
3088
3089
3090
3091
3092
3093
3094
3095
3096
3097
3098
3099
3100
3101
3102
3103
3104
3105
3106
3107
3108
3109
3110
3111
3112
3113
3114
3115
3116
3117
3118
3119
3120
3121
3122
3123
3124
3125
3126
3127
3128
3129
3130
3131
3132
3133
3134
3135
3136
3137
3138
3139
3140
3141
3142
3143
3144
3145
3146
3147
3148
3149
3150
3151
3152
3153
3154
3155
3156
3157
3158
3159
3160
3161
3162
3163
3164
3165
3166
3167
3168
3169
3170
3171
3172
3173
We cannot predict
3174
Continuing
3175
3176
3177
3178

In [ ]:
# print("unpredicted = ",unpredicted_count[1])
print(len(predicted[1]))
threshold = 7
ttest = test[1].overall
list1 = replace_with_zero_one(ttest.tolist(), threshold)
list2 = replace_with_zero_one(predicted[1], threshold)
# mean average error
print("mae = ",mean_average_error(ttest, predicted[1]))
# precision
print("precision = ",precision(list1, list2, threshold))
# recall
print("recall = ",recall(list1, list2, threshold))
# f1 score
print("f1-score = ",f1_score(list1, list2, threshold))